In [199]:
import pandas as pd
import numpy as np

In [200]:
demo = pd.read_csv('data/demo.csv',index_col='ip_id')
y_train = pd.read_csv('data/y_train.csv',index_col='ip_id')

# Preprocess Demo

In [201]:
demo.head(1)

,brth_yr,act_strt_dt,no_of_dpnd_chl,cis_income,crn_bal,gnd_cd,mar_st_cd,ctf_tp_cd,ocp_cd
ip_id,,,,,,,,,
1,1990.0,2011-12-11,0.0,45864.0,876428.0,2.0,2.0,3.0,9


In [202]:
## Create Feature Age when opened the account
demo['age_open_acc'] = demo['act_strt_dt'].str.split('-').str.get(0).apply(float) - demo['brth_yr']

# Normalize

In [203]:
demo = demo[['brth_yr','age_open_acc','cis_income','crn_bal','no_of_dpnd_chl','ctf_tp_cd']]

In [204]:
demo_train = demo[demo.index.isin(pd.read_csv('data/y_train.csv')['ip_id'])]
demo_train.shape

(6361, 6)

In [205]:
demo_test = demo[demo.index.isin(pd.read_csv('data/y_train.csv')['ip_id']) == False]
demo_test.shape

(3938, 6)

In [206]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
minmaxscaler = scaler.fit(demo_train)
demo_train_scaled = minmaxscaler.transform(demo_train)
demo_test_scaled = minmaxscaler.transform(demo_test)

# Model

In [207]:
label = y_train['label'].values
features = demo_train_scaled

In [208]:
label.shape

(6361,)

In [209]:
features.shape

(6361, 6)

In [210]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(features, label) 


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')

# Predict 

In [211]:
neigh.predict(demo_test_scaled)

array([0., 0., 0., ..., 1., 1., 1.])

In [212]:
neigh.predict_proba(demo_test_scaled)[:,1]

array([0.  , 0.  , 0.  , ..., 0.75, 1.  , 1.  ])

In [213]:
neigh.predict_proba(predict_feature)[:,1].shape

(3938,)

In [214]:
demo_test['prob1'] = neigh.predict_proba(predict_feature)[:,1]

/home/thanatlapthawan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Chose only 281 group

In [215]:
demo_test = demo_test[(demo_test.index.isin(pd.read_csv('data/y_test_index.csv',index_col='ip_id').index)) & \
                  (demo_test.index.isin(pd.read_csv('data/cc_txn.csv',index_col='ip_id').index.unique())== False) &
                 (demo_test.index.isin(pd.read_csv('data/sa.csv',index_col='ip_id').index.unique())== False)]
demo_test = demo_test[['prob1']]

In [217]:
demo_test[['prob1']].to_csv('result_281.csv')